# RunDown of our workshop

* Quick introduction to essential parts of tensorflow
* Neural network essentials
* Neural network training code
* Transfer learning



# Tensorflow essentials
Tensorflow is low level library, but there exists high level interfaces that help us to do faster prototyping of ideas. If you want you can check them on your own
* TF Learn (tf.contrib.learn): simplified interface that helps users transition from the the world of one-liner such as scikit-learn
* TF Slim (tf.contrib.slim): lightweight library for defining, training and evaluating complex models in TensorFlow.
* High level API: Keras, TFLearn, Pretty Tensor

In this tutorial we will be using native tensorflow and slim. In future Keras will be fully integrated to tensorflow as parts of slim has been integrated from contrib to core.

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

# Tensors

The central unit of data in TensorFlow is the tensor. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's rank is its number of dimensions. Here are some examples of tensors:

```python
3 # a rank 0 tensor; this is a scalar with shape []
[1. ,2., 3.] # a rank 1 tensor; this is a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]
```
You can look at tensors as an n-dimensional matrix
* 0-d tensor: scalar (number)
* 1-d tensor: vector
* 2-d tensor: matrix
* and so on 


In [3]:
#As you see interface is very similar to numpy.
a = np.random.randint(1, 10, size=3)
print a, a.shape
a = np.random.randint(1, 10, size=(2,3))
print a, a.shape
a = np.random.randint(1, 10, size=(2,1,3))
print a, a.shape
#Tensorflow computation primitives mimic numpy funtions
print tf.divide
print np.divide

[4 5 6] (3,)
[[1 3 6]
 [4 4 5]] (2, 3)
[[[2 1 3]]

 [[1 9 4]]] (2, 1, 3)
<function divide at 0x7fd5b437eaa0>
<ufunc 'divide'>


# The Computational Graph

You might think of TensorFlow Core programs as consisting of two discrete sections:

1. Building the computational graph.
2. Running the computational graph.

A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Let's build a simple computational graph. When not stated explicitly with tf.Graph() - all defined operations is stored in default graph.

In [15]:
a = tf.add(2,3)
print a
b = np.add(2,3)
print b

Tensor("Add_2:0", shape=(), dtype=int32)
5


# Session  
tf.Session() -  A Session object encapsulates the environment in which **Operation objects** (add) are
executed, and **Tensor objects**(2 and 3) are evaluated.


In [19]:
x = 2
y = 3
op1 = tf.add(x, y)
op2 = tf.multiply(x, y)

#What is value of useless? Why?
useless = tf.multiply(x, op1)
op3 = tf.pow(op2, op1)
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.05)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    print sess.run(op3)
    
#     We can evalue different subgraphs simultaneously
#     print sess.run([op3, useless])
    

7776
[7776, 10]


# Constants and variables
Lets explore tensor objecs which are **constants** and **variables**



In [31]:
a = tf.constant(2, name="a")
print a
b = tf.constant([2,2], name="b")
print b
c = tf.constant(np.random.randint(1,10, 10), name="c")
print c


Tensor("a_9:0", shape=(), dtype=int32)
Tensor("b_3:0", shape=(2,), dtype=int32)
Tensor("c_2:0", shape=(10,), dtype=int64)


```python
tf.linspace(10.0, 13.0, 4) ==> [10.0 11.0 12.0 13.0]
tf.range(start, limit, delta) ==> [3, 6, 9, 12, 15]
```
But you cant do for _ in tf.range(10) as tensor object are not iterable
```python
tf.random_uniform((3,3))
tf.random_normal((5,5))
tf.int32 == np.int32 # True
```
As you see interface is very similar to numpy or python

```python
# create variable a with scalar value
a = tf.Variable(2, name="scalar")
# create variable b as a vector
b = tf.Variable([2, 3], name="vector")
# create variable c as a 2x2 matrix
c = tf.Variable([[0, 1], [2, 3]], name="matrix")
# create variable W as 784 x 10 tensor, filled with zeros
W = tf.Variable(tf.zeros([784,10]))
```


**You always have to initialize your variables!!!**


In [40]:
b = tf.Variable([2, 3], name="vector")
print b
# b.eval() # try it 

init_op  = tf.global_variables_initializer()
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.05)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    sess.run(init_op)
    #you can evaluate using sess.run(b) or b.eval() if you have active session
    print b.eval()

Tensor("vector_4/read:0", shape=(2,), dtype=int32)
[2 3]


# Code challenge
* What value print function will print and why?

In [4]:
W = tf.Variable(10)
W.assign(100)
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.05)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    sess.run(W.initializer)
    #print W.eval() # >> What value do you get and why?
    

# Placeholders
Imagine you want to calculate function y = x*2 + b, but you want to suply x values on runtime. 

Placeholders let you inject data directly to computation grahp. Feed values to placeholder directly using dictionary

In [49]:
# create a placeholder of type float 32-bit, shape is a vector of any number of elements
x = tf.placeholder(tf.float32, shape=None)
# create a constant of type float 32-bit, shape is a vector of 1 elements
b = tf.constant(5, tf.float32)
# use the placeholder as you would a constant or a variable
y = 2*x + b # 
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.05)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
# feed [1, 2, 3] to placeholder a via the dict {a: [1, 2, 3]}
# fetch value of 
    print sess.run(y, {x: [2]})
    print sess.run(y, {x: [2,3]})

[ 9.]
[  9.  11.]


# The trap of lazy loading*
Defer creating/initializing an object until it is needed. The non bug that i personaly commited:). Lets examine some code

* The good

```python
    x = tf.Variable(10, name='x')
    y = tf.Variable(20, name='y')
    z = tf.add(x, y) # you create the node for add node before executing the graph
    gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.05)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
        sess.run(tf.global_variables_initializer())
        for _ in range(10):
            sess.run(z)
```

* The bad

```python
    x = tf.Variable(10, name='x')
    y = tf.Variable(20, name='y')
    gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.05)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
        sess.run(tf.global_variables_initializer())
        for _ in range(10):
            sess.run(tf.add(x,y))
```

1. and no ugly, but what is the source of the problem?


# Lets build first model